# Base64 이미지 태그 → 파일 경로 변환 (v2)

특정 sourceId의 문서에서 base64 인코딩된 이미지 태그를 파일 경로 참조로 변환합니다.

In [1]:
import re

# 설정
INPUT_FILE = r"C:\Users\LEEJUHWAN\퇴사자명단\documents_all.cmds"
OUTPUT_FILE = r"C:\Users\LEEJUHWAN\퇴사자명단\documents_all_converted.cmds"

# 처리 대상 sourceId 목록
TARGET_SOURCE_IDS = {
    "2978801", "2983943", "2985244", "3091219", "3097823", "3120072",
    "3187289", "3190258", "3202897", "3271155", "3288192", "3307760",
    "5637771", "5641483", "5649101", "5810171"
}

print(f"처리 대상 sourceId 개수: {len(TARGET_SOURCE_IDS)}")

처리 대상 sourceId 개수: 16


In [2]:
# 파일 구조 먼저 확인 (처음 몇 줄만)
with open(INPUT_FILE, 'r', encoding='utf-8') as f:
    for i, line in enumerate(f):
        if i < 3:  # 처음 3줄만
            print(f"줄 {i+1} (길이: {len(line)}): {line[:100]}...")
        else:
            break

줄 1 (길이: 44734): addDocument {"sourceId":"15167823","docNum":"ANY-사업-791","references":[],"docType":"DRAFT","title":"...
줄 2 (길이: 26215): addDocument {"sourceId":"15168441","docNum":"ANY-사업수행-482","references":[],"docType":"DRAFT","title"...
줄 3 (길이: 19344): addDocument {"sourceId":"15162004","docNum":"ANY-경비627","references":[],"docType":"DRAFT","title":"출...


In [3]:
# 통계
stats = {
    'total_lines': 0,
    'target_docs_found': 0,
    'docs_with_base64': 0,
    'docs_with_multiple_images': [],
    'converted': 0
}

# base64 이미지 태그 패턴
base64_img_pattern = r'<img[^>]+data:image/[^>]+>'

print("설정 완료")

설정 완료


In [4]:
# 변환 실행
print("변환 시작...\n")

output_lines = []

with open(INPUT_FILE, 'r', encoding='utf-8') as f:
    for line in f:
        stats['total_lines'] += 1
        
        # sourceId 추출 시도
        source_id_match = re.search(r'"sourceId"\s*:\s*"(\d+)"', line)
        
        if source_id_match:
            source_id = source_id_match.group(1)
            
            # 대상 sourceId인지 확인
            if source_id in TARGET_SOURCE_IDS:
                stats['target_docs_found'] += 1
                
                # base64 이미지 찾기
                base64_images = re.findall(base64_img_pattern, line)
                
                if base64_images:
                    stats['docs_with_base64'] += 1
                    
                    # 이미지 2개 이상이면 알림
                    if len(base64_images) > 1:
                        stats['docs_with_multiple_images'].append({
                            'sourceId': source_id,
                            'count': len(base64_images)
                        })
                        print(f"⚠️  sourceId {source_id}: base64 이미지 {len(base64_images)}개 발견!")
                    
                    # 변환
                    new_tag = f'<img src="/PMS_SITE-U7OI43JLDSMO/approval/attach/approval_2020_img/apr{source_id}/0.jpg"/>'
                    line = re.sub(base64_img_pattern, new_tag, line)
                    
                    stats['converted'] += 1
                    print(f"✅ sourceId {source_id}: 변환 완료")
        
        output_lines.append(line)

print(f"\n처리 완료! 총 {stats['total_lines']:,}줄 처리")

변환 시작...

✅ sourceId 2985244: 변환 완료
✅ sourceId 3091219: 변환 완료
✅ sourceId 3097823: 변환 완료
✅ sourceId 3120072: 변환 완료
✅ sourceId 3187289: 변환 완료
✅ sourceId 3190258: 변환 완료
✅ sourceId 3202897: 변환 완료
✅ sourceId 3271155: 변환 완료
✅ sourceId 3288192: 변환 완료
✅ sourceId 3307760: 변환 완료
✅ sourceId 5637771: 변환 완료
✅ sourceId 5641483: 변환 완료
✅ sourceId 5649101: 변환 완료
✅ sourceId 5810171: 변환 완료
✅ sourceId 2983943: 변환 완료

처리 완료! 총 23,320줄 처리


In [5]:
# 결과 저장
with open(OUTPUT_FILE, 'w', encoding='utf-8') as f:
    f.writelines(output_lines)

print(f"저장 완료: {OUTPUT_FILE}")

저장 완료: C:\Users\LEEJUHWAN\퇴사자명단\documents_all_converted.cmds


In [6]:
# 통계 출력
print("\n" + "="*50)
print("처리 결과 통계")
print("="*50)
print(f"전체 줄 수: {stats['total_lines']:,}")
print(f"대상 sourceId 문서 수: {stats['target_docs_found']}")
print(f"base64 이미지 포함 문서: {stats['docs_with_base64']}")
print(f"변환 완료: {stats['converted']}")

if stats['docs_with_multiple_images']:
    print(f"\n⚠️  이미지 2개 이상인 문서:")
    for doc in stats['docs_with_multiple_images']:
        print(f"   - sourceId {doc['sourceId']}: {doc['count']}개")
else:
    print(f"\n✅ 모든 대상 문서가 이미지 1개 이하입니다.")


처리 결과 통계
전체 줄 수: 23,320
대상 sourceId 문서 수: 16
base64 이미지 포함 문서: 15
변환 완료: 15

✅ 모든 대상 문서가 이미지 1개 이하입니다.
